# Compte rendu de TP <br/>
Régression linéaire

## Rappel des rèlges de notation

Le TP de chaque étudiant sera évalué via la plateforme “Classgrade”. Pour cela, vous devez déposer
un unique fichier anonymisé (votre nom doit apparaître uniquement dans le nom du fichier lui-même)
sous format ipynb sur le site http://peergrade.enst.fr/.<br/>
Vous devez **charger votre fichier, avant le jeudi 18/09/2018, 23h59**. Entre le vendredi 19/09/2018 et
**le vendredi 26/09/2018, 23h59, vous devrez noter 2 copies** qui vous seront assignées anonymement, en
tenant compte du barème suivant pour chaque question du TP :
 * 0 (manquant/ non compris/ non fait/ insuffisant)
 * 1 (passable/partiellement satisfaisant)
 * 2 (bien)
 
Ensuite, il faudra également évaluer de la même manière les points suivants (qui correspondent à 3
questions supplémentaires) :
 * aspect global de présentation : qualité de rédaction, orthographe, présentation, graphes, titres,
etc...
 * aspect global du code : indentation, Style PEP8, lisibilité du code, commentaires adaptés
 * Point particulier : absence de bug sur votre machine
 
Des commentaires pourront être ajoutés question par question si vous en sentez le besoin ou l’utilité pour
aider la personne notée à s’améliorer, et de manière obligatoire si vous ne mettez pas 2/2 à une question.
**Enfin, veillez à rester polis et courtois dans vos retours.**

## Exercice:

### Sujet

(Analyse de la base de données “investment data”) La lecture d’un tutoriel
pandas pourra être utile : http://pandas.pydata.org/pandas-docs/stable/tutorials.html.
Nous travaillons sur la base de données Investment Data Set 1 qui peut être téléchargée depuis
https://bitbucket.org/portierf/shared_files/downloads/invest.txt.<br/> Avant de commencer, on
réalisera l’exercice 12 du polycopié : ”explicit formulas when p “ 1 for prediction intervals", se trouvant
dans le chapitre 3 : “Confidence intervals and hypothesis testing”. On pourra aussi lire la section 3.1.1 de
ce même chapitre du polycopié.

In [58]:
#Cellule d'import 

#pandas
import pandas as pd
#graph'
import numpy as np
import math
import plotly.plotly as py
import plotly.graph_objs as go

### Question 1

Importer la base de données “invest.txt” et l’afficher dans une forme lisible, e.g. une table contenant
les 5 premières observations.


In [59]:
investDF = pd.read_csv("https://bitbucket.org/portierf/shared_files/downloads/invest.txt", header=0,sep=" ")
print(investDF.head())

   year     gnp  invest     cpi  interest
0  1968   873.4   133.3   82.54      5.16
1  1969   944.0   149.3   86.79      5.87
2  1970   992.7   144.2   91.45      5.95
3  1971  1077.6   166.4   96.01      4.88
4  1972  1185.9   195.0  100.00      4.50


In [60]:
#Petit apperçu des données
investDF.describe(include='all')      

,year,gnp,invest,cpi,interest
count,15.000000,15.000000,15.000000,15.000000,15.000000
mean,1975.000000,1748.646667,276.006667,131.401333,7.452667
std,4.472136,738.145808,117.582691,40.286606,2.812245
min,1968.000000,873.400000,133.300000,82.540000,4.500000
25%,1971.500000,1131.750000,180.700000,98.005000,5.480000
50%,1975.000000,1549.200000,229.800000,125.790000,6.250000
75%,1978.500000,2290.850000,394.450000,156.920000,9.055000
max,1982.000000,3057.500000,471.500000,207.230000,13.420000


### Question 2

Réaliser le graph suivant : la variable “Gross National Product” (GNP, column “gnp”) est en abscisse
et la variable “Investment” (column “invest”) est en ordonnée. Transformer les 2 variables précédentes
en échelle logarithmique. Nous travaillerons désormais avec les 2 nouvelles variables.

In [61]:
# Changement d'echelle
investDF['invest']= investDF.invest.apply(lambda x: math.log(x))
investDF['gnp']= investDF.gnp.apply(lambda x: math.log(x))

In [62]:
data = [go.Scatter(x=investDF.gnp,
               y=investDF.invest)]
layout = go.Layout(title="Evolution of Investment depending on Gross National Produc",
                   yaxis=dict(title='Investment (log)'),
                   xaxis=dict(title='Gross National Produict (log)'))
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='jupyter-basic_bar')

In [63]:
# Si affichage ci-dessus non fonctionnel
#plt.plot(investDF.gnp,investDF.invest)
#plt.xlabel('Gross National Produc')
#plt.ylabel('Investment')
#plt.show()

NOTE : Lorsque l’on traite des données monétaires, on travaille souvent en échelle logarithmique (pour
prendre en compte les différences d’échelle).

### Question 3

Pour la régression de ”Investment” (variable à expliquer, output) sur ”GNP” (variable explicative,
covariable), estimer l’intercept et la pente, leurs écart-types, ainsi que le coefficient de détermination.
Les afficher dans une forme lisible. Dans la suite le vecteur contenant l’intercept et la pente est noté $\hat{\theta_n}$


In [64]:
n=investDF.shape[0]
Ymean=investDF.invest.mean() 
Xtildemean=investDF.gnp.mean() 

Xtilde=np.array(investDF.gnp.values)
Y=np.array(investDF.invest.values)

#Calcul de ma pente
beta1=np.dot(np.transpose(Xtilde-Xtildemean),(Y-Ymean))/np.dot(np.transpose(Xtilde-Xtildemean),(Xtilde-Xtildemean))
print('beta1 estimé = '+ str(beta1)+" (correspond à la pente)")

#Calcul de l'intercept
beta0=Ymean - beta1 * Xtildemean
print('beta0 estimé =' + str(beta0)+" (correspond à l'intercept)")



#Calcul du coefficient de determination
Yestime=beta0+beta1*Xtilde
Rsquare=(np.dot(np.transpose(Yestime-Ymean),Yestime-Ymean)/np.dot(np.transpose(Y-Ymean),Y-Ymean))
print("Le coéficient de determination vaut "+str(Rsquare))

Testime=np.array([beta0,beta1])

# Calcul de sigma estime
from numpy.linalg import matrix_rank
X=np.array([np.ones(n),Xtilde]).T
sigma_estime=np.sqrt(np.dot(np.transpose(Y-np.dot(X,Testime)),Y-np.dot(X,Testime))/(n-matrix_rank(X)))


#Calculs des ecarts types
Vbeta0=(sigma_estime**2)*((1/n)+(Xtildemean*Xtildemean/np.dot(np.transpose(Xtilde-Xtildemean),Xtilde-Xtildemean)))
Vbeta1=(sigma_estime**2)*((1/np.dot(np.transpose(Xtilde-Xtildemean),Xtilde-Xtildemean)))

print("L'écart type de beta0 vaut "+str(np.sqrt(Vbeta0)))
print("L'écart type de beta1 vaut "+str(np.sqrt(Vbeta1)))

beta1 estimé = 1.015281401598871 (correspond à la pente)
beta0 estimé =-1.9635913352301815 (correspond à l'intercept)
Le coéficient de determination vaut 0.9627572956057854
L'écart type de beta0 vaut 0.4095472716623086
L'écart type de beta1 vaut 0.055383060354852874


In [65]:
trace1=go.Scatter(x=investDF.gnp,
            y=investDF.invest,
            name="Original")
trace2=go.Scatter(x=Xtilde,
            y=Yestime,
            name="Estimé")

data = [trace1,trace2]
#type='log',autorange=True
layout = go.Layout(title="Estimation VS Realité",
                   yaxis=dict(title='Investment (log)'),
                   xaxis=dict(title='Gross National Produict (log)'))
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='jupyter-basic_bar')

### Question 4

La pente estimée précédemment est-elle statistiquement significative ? On fera un test de student
(t-test). Donner la valeur de la statistique de test ainsi que la p-valeur.

On pose $H_0$ l'hypothese que la pente estimée n'est pas statistiquement significative soit $\beta_1 = 0$ <br/>
Des lors l'hypothèse alternative peut etre défini comme $H_1 \Rightarrow \beta_1 \ne 0$

In [66]:
from scipy import stats as scs
Tj= beta1/(np.sqrt(Vbeta1))
p = matrix_rank(X)*(1-scs.t.cdf(Tj, X.shape[0] - matrix_rank(X)))
print('La p-value vaut :'+str(p))

La p-value vaut :1.1363709973011282e-10


A partir de la $p-value$ on peut rejeter l'hypothese $H_0$

### Question 5
Pour GNP égal à 1000, estimer l’investissement prédit par le modèle. Pour GNP égal à 1000, donner l’intervalle de confiance pour la valeur prédite et l’intervalle de confiance pour la variable à expliquer “Investment”, au niveau 90%. On pourra se référer à la section 3.1.3 “Confidence intervals for the predicted values” du polycopié dans laquelle chaque intervalle est défini, $CI(x)$ et $PI(x)$, respectivement (avec les notations du polycopié, $x=(1, 1000)^T$ )

In [67]:
def predY(x,Teta):
    XX=np.array([1]+x)
    return np.dot(np.transpose(XX),(Teta))


Y1000=predY([np.log(1000)],Testime)
print("Pour GNP égal à 1000 l’investissement prédit ( en réel ) vaut "+str(np.exp(Y1000)))
print("Pour GNP égal à 1000 l’investissement prédit ( en log ) vaut "+str(Y1000))

Pour GNP égal à 1000 l’investissement prédit ( en réel ) vaut 155.97942793105494
Pour GNP égal à 1000 l’investissement prédit ( en log ) vaut 5.049724126316802


En ayant $H_1 \Rightarrow \beta_1 \ne 0$, on suppose qu'on ne peut connaitre le signe de $\beta_1$ et nous nous trouvons dans un cas de bilatéralité

In [68]:
import numpy as np
from scipy.stats import norm
def getICIP(alpha,XX,X,Teta,sig):
    
    x = np.array([1]+XX)
    q = scs.t.ppf(1 - alpha/2, X.shape[0] - matrix_rank(X))
    Cinf = np.dot(np.transpose(x),(Teta)) - sig * q * np.sqrt(np.dot(np.dot(np.transpose(x),np.linalg.inv(np.dot(np.transpose(X),X))),x))
    Csup = np.dot(np.transpose(x),(Teta)) - sig * (1 - q) * np.sqrt(np.dot(np.dot(np.transpose(x),np.linalg.inv(np.dot(np.transpose(X),X))),x))
    

    Pinf = np.dot(np.transpose(x),(Teta)) - sig * q * np.sqrt(1+np.dot(np.dot(np.transpose(x),np.linalg.inv(np.dot(np.transpose(X),X))),x))
    Psup = np.dot(np.transpose(x),(Teta)) - sig * (1 - q) * np.sqrt(1+np.dot(np.dot(np.transpose(x),np.linalg.inv(np.dot(np.transpose(X),X))),x))
    
    return Cinf,Csup,Pinf,Psup

ICinf,ICsup,PIinf,PIsup=getICIP(.1,[np.log(1000)],X,Testime,sigma_estime)
print('IC = [' + str(ICinf)+',' + str(ICsup) + ']')
print('PI = [' + str(PIinf)+',' + str(PIsup) + ']')



IC = [4.988222845971108,5.076497234228081]
PI = [4.88282002343868,5.122381825995147]


### Question 6
Sur un graphe avec échelle logarithmique, avec GNP en abscisse et investment en ordonnée, tracer les données, la droite de régression, ainsi que les intervalle CI et PI (pour toutes les valeurs de log(GNP) comprises entre le maximum et le minimum observé sur les données)

In [69]:
YPRED=[]
ICSUP=[]
ICINF=[]
IPSUP=[]
IPINF=[]

XX=np.linspace(Xtilde.min(), Xtilde.max(), 500)

for x in XX:
    YPRED.append(predY([x],Testime))
    ICinf_tmp,ICsup_tmp,PIinf_tmp,PIsup_tmp=getICIP(.1,[x],X,Testime,sigma_estime)
    ICSUP.append(ICsup_tmp)
    ICINF.append(ICinf_tmp)
    IPSUP.append(PIsup_tmp)
    IPINF.append(PIinf_tmp)
    


trace0 = go.Scatter(
    x=XX[0::2],
    y=ICSUP[0::2],
    fill='tonexty',
    fillcolor='rgba(255, 215, 108,0.4)',
    line=dict(color='rgba(255,255,255,0)'),
    name='ICSUP',
    showlegend=True,
)
trace00 = go.Scatter(
    x=XX[0::2],
    y=ICINF[0::2],
    line=dict(color='rgba(255,255,255,0)'),
    name='ICINF',
    showlegend=False,
)
traceA = go.Scatter(
    x=XX[1::2],
    y=IPSUP[1::2],
    line=dict(color='rgba(255,255,255,0)'),
    fillcolor='rgba(0,176,246,0.2)',
    fill='tonexty',
    name='IPSUP',
    showlegend=True,
)
traceAA = go.Scatter(
    x=XX[1::2],
    y=IPINF[1::2],
    line=dict(color='rgba(255,255,255,0)'),
    name='IPINF',
    showlegend=False,
)

trace1=go.Scatter(x=XX,
            y=YPRED,
            name="Estimé")
trace2=go.Scatter(x=Xtilde,
            y=Y,
            name="Original")

data = [traceAA,traceA,trace00,trace0,trace1,trace2]
#type='log',autorange=True
layout = go.Layout(title="Estimation VS Realité",
                   yaxis=dict(title='Investment (log)'),
                   xaxis=dict(title='Gross National Produict (log)'))
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='jupyter-basic_bar')

### Querstion 7
En utilisant des classes/librairies existantes, donner l’intercept, la pente, le coefficient de dé- termination ainsi que l’investissement prédit par le modèle quand GNP vaut 100. La classe **LinearRegression() de sklearn.linear_model** est suggérée mais pas obligatoire. Vérifier que les valeurs calculées ici coïncident avec celles des questions précédentes.

In [70]:
from sklearn.linear_model import LinearRegression
regr = LinearRegression()
# Train the model using the training sets
regr.fit(Xtilde.reshape(-1, 1), Y)

# Make predictions using the testing set
regr.coef_

print("Intercept des librairies : "+str(regr.intercept_) +" Intercept calculé : "+str(beta0))
print("Pente des librairies : "+str(regr.coef_[0]) +" pente calculé : "+str(beta1))
print("Coefficient de détermination des librairies : "+str(regr.score(Xtilde.reshape(-1, 1), Y)) +" Coefficient de détermination calculé : "+str(Rsquare))
print("Y(1000) prédit des librairies : "+str(regr.predict([[np.log(1000)]])[0]) +" Y(1000) prédit calculé : "+str(Y1000))


Intercept des librairies : -1.9635913352301815 Intercept calculé : -1.9635913352301815
Pente des librairies : 1.015281401598871 pente calculé : 1.015281401598871
Coefficient de détermination des librairies : 0.9627572956057855 Coefficient de détermination calculé : 0.9627572956057854
Y(1000) prédit des librairies : 5.049724126316802 Y(1000) prédit calculé : 5.049724126316802


### Question8
Sur un graph avec échelle logarithmique, avec GNP en abscisse et investment en ordonnée, tracer les données, la droite de régression, ainsi que l’investissement prédit par le modèle quand GNP vaut 100 (on donnera à ce point une couleur différente).

In [71]:

YPREDlib=regr.predict(XX.reshape(-1, 1))
trace1=go.Scatter(x=XX,
            y=YPREDlib,
            name="Estimé")
trace2=go.Scatter(x=Xtilde,
            y=Y,
            name="Original")
trace3=go.Scatter(x=[np.log(1000)],
            y=regr.predict([[np.log(1000)]]),
            mode = 'markers',
            marker=dict(color='rgba(201, 29, 46,1)',
                       size=15),
            showlegend=False)

data = [trace1,trace2,trace3]
#type='log',autorange=True
layout = go.Layout(title="Etude des valeurs prédites",
                   yaxis=dict(title='Investment (log)'),
                   xaxis=dict(title='Gross National Produict (log)'))
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='jupyter-basic_bar')

### Question 9
Pour la régression de Investment sur GNP et interest, calculer la matrice de Gram. Est-elle de rang plein ?

In [107]:
n=investDF.shape[0]
X=np.array([np.ones(n),investDF.gnp,investDF.interest]).T
Xmean=np.array([1,investDF.gnp.mean(),investDF.interest.mean()]).T
mGram=np.dot(X.T,X)
print(mGram)
print("Le rang de la matrice de gram est "+ str(matrix_rank(mGram)))

[[ 15.         110.75282116 111.79      ]
 [110.75282116 820.24847688 839.18854302]
 [111.79       839.18854302 943.8557    ]]
Le rang de la matrice de gram est 3


La matrice de Gram est effectivement de rang plein

### Question 10

Pour la régression de Investment sur GNP et interest, estimer les 3 coefficients et leurs écart- types ainsi que le coefficient de détermination. En plus, faire un test de Student de significativité de chaque coefficient (donner la statistique de test et la p-valeur). Afficher les résultats dans une forme convenable. Discuter de la significativité des coefficients.

In [122]:

Ymean=investDF.invest.mean() 
Y=np.array(investDF.invest.values)


TetaEstime=np.dot(np.linalg.inv(mGram),np.dot(X.T,Y))
print(TetaEstime)
            

#Calcul du coefficient de determination
Yestime=np.dot(X,TetaEstime)
Rsquare=(np.dot(np.transpose(Yestime-Ymean),Yestime-Ymean)/np.dot(np.transpose(Y-Ymean),Y-Ymean))
print("Le coéficient de determination vaut "+str(Rsquare))

sigma_estime=np.sqrt(np.dot(np.transpose(Y-np.dot(X,TetaEstime)),Y-np.dot(X,TetaEstime))/(n-matrix_rank(X)))
print(sigma_estime)
#Calculs des ecarts types
covTeta=(sigma_estime**2)*np.linalg.inv(mGram)
print(covTeta)

print("L'écart type de beta0 vaut "+str(np.sqrt(covTeta[0][0])))
print("L'écart type de beta1 vaut "+str(np.sqrt(covTeta[1][1])))
print("L'écart type de beta1 vaut "+str(np.sqrt(covTeta[2][2])))

[-2.18045473  1.05124269 -0.00652888]
Le coéficient de determination vaut 0.9633108306727456
0.09051205972421447
[[ 4.38772697e-01 -6.72457405e-02  7.82053357e-03]
 [-6.72457405e-02  1.04165249e-02 -1.29683702e-03]
 [ 7.82053357e-03 -1.29683702e-03  2.35444629e-04]]
L'écart type de beta0 vaut 0.6623991977963686
L'écart type de beta1 vaut 0.10206137806192304
L'écart type de beta1 vaut 0.01534420506560419


### Question 11
Pour les valeurs de GNP 1000 et interest 10, i.e., $x = (1, 1000, 10)^T$ , prédire log(investment) et donner les intervalles de confiance $CI(x)$ et $PI(x)$ au niveau 99.9%

In [121]:
ICinf,ICsup,PIinf,PIsup=getICIP(.01,[np.log(1000),10],X,TetaEstime,sigma_estime)
print('IC = [' + str(ICinf)+',' + str(ICsup) + ']')
print('PI = [' + str(PIinf)+',' + str(PIsup) + ']')

IC = [4.75013136139408,5.194800930638004]
PI = [4.6324284942872165,5.273970047009964]


### Question 12
Sur un même graph à 3 dimensions avec les axes suivants : log(GNP), Interest, and log(Investment), tracer les données, le “plan” de régression et les surfaces correspondantes aux intervalles de confiance à 99.9% (ces surfaces seront tracées sur le domaine de définition des données).

### Question 13

En utilisant des classes/librairies existantes, donner les coefficients de régression, le coefficient de détermination ainsi que l’investissement prédit par le modèle quand GNP vaut 100 et interest 10. La classe LinearRegression() de sklearn.linear_model est suggérée mais pas obligatoire. Vérifier que les valeurs calculées ici coïncident avec celles des questions précédentes.


In [129]:
from sklearn.linear_model import LinearRegression
regr = LinearRegression()
# Train the model using the training sets
Xtilde=np.array([investDF.gnp,investDF.interest]).T
print(Xtilde)
regr.fit(Xtilde, Y)

# Make predictions using the testing set
regr.coef_

print("beta0 des librairies : "+str(regr.intercept_) +" Intercept calculé : "+str(TetaEstime[0]))
print("Pente des librairies : "+str(regr.coef_[0]) +" pente calculé : "+str(TetaEstime[1]))
print("Pente des librairies : "+str(regr.coef_[1]) +" pente calculé : "+str(TetaEstime[2]))
print("Coefficient de détermination des librairies : "+str(regr.score(Xtilde, Y)) +" Coefficient de détermination calculé : "+str(Rsquare))
print("Y(1000,10) prédit des librairies : "+str(regr.predict([[np.log(1000),10]])[0]) +" Y(1000,10) prédit calculé : "+str(Y1000))


[[ 6.77239364  5.16      ]
 [ 6.85012617  5.87      ]
 [ 6.9004285   5.95      ]
 [ 6.98249163  4.88      ]
 [ 7.07825726  4.5       ]
 [ 7.19022378  6.44      ]
 [ 7.26836248  7.83      ]
 [ 7.34549395  6.25      ]
 [ 7.4489161   5.5       ]
 [ 7.55919466  5.46      ]
 [ 7.67966743  7.46      ]
 [ 7.79061331 10.28      ]
 [ 7.87591714 11.77      ]
 [ 7.98538224 13.42      ]
 [ 8.02535287 11.02      ]]
beta0 des librairies : -2.1804547259114004 Intercept calculé : -2.180454725912682
Pente des librairies : 1.0512426913196595 pente calculé : 1.0512426913200557
Pente des librairies : -0.006528879406068252 pente calculé : -0.006528879406129562
Coefficient de détermination des librairies : 0.9633108306726245 Coefficient de détermination calculé : 0.9633108306727456
Y(1000,10) prédit des librairies : 5.0159837304826835 Y(1000,10) prédit calculé : 5.049724126316802
